In [1]:
# Use a pipeline as a high-level helper
import pdb
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration, BitsAndBytesConfig
import torch
from PIL import Image
import time
from llavaAgent import LLaVaAgent


In [2]:

agent = LLaVaAgent('mistral', '7b')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import cv2
im = cv2.imread("images_for_vlm/scene1.png")
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
pt = ("You are a robot navigating within an 3-D environment as shown. In the image you see, there are "
                  f"{4} labeled objects. You will be asked to analyze the spatial position of these labeled "
                  f"objects with relation to each other. The red dots on each object are the object's center of mass, "
                  f"which you should use when comparing the position of two objects. From your point of view, "
                  f"answer each question with the"
                  f"descriptors right/left, above/below, in front/behind. If there is no clear spatial difference "
                  f"along a given axis, you can answer 'neutral'")
pt += ("\n1. Where is the led tv in relation to the couch?\n2. Where is the couch in relation to the chandelier?\n3. Where is the chandelier in relation to the kitchen countertop item?")

pt +=  ("\nReason through the task  and describe the 3d layout of the image you see. Output your thinking and at the very end output "
                   "'ANSWER' followed by a json object in the following example format:"
                   "\n{1: ['right', 'above', 'neutral'], 2: ['left', 'neutral', 'in front']}\nThere should be 3 key-value pairs corresponding to the 3 questions")

out, eff = agent.call(im, pt, 3)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


starting output
llava-v1.6-mistral-7b-hf finished inference, took 21.0825252532959 seconds


In [6]:
print(out)

To analyze the spatial position of the labeled objects, I will first describe the 3D layout of the image:

- The kitchen countertop item is located in the foreground, on the left side of the image.
- The couch is situated in the background, on the right side of the image.
- The chandelier is hanging from the ceiling, directly above the kitchen countertop item.
- The led tv is mounted on the wall, to the right of the couch.

Now, let's analyze the spatial position of these objects:

1. Where is the led tv in relation to the couch?
   - The led tv is to the right of the couch.
   - Since the couch is in the background and the led tv is mounted on the wall, they are not directly above or below each other.
   - The led tv is in front of the couch, as it is mounted on the wall and the couch is on the floor.
   ANSWER: {1: ['right', 'neutral', 'in front']}

2. Where is the couch in relation to the chandelier?
   - The couch is to the left of the chandelier.
   - The chandelier is hanging fro

In [18]:
out.index('{')



In [22]:

d[1][0]

'right'

In [13]:

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf",
                                                          quantization_config=quantization_config,
                                                          device_map="auto",
                                                          torch_dtype=torch.float16, low_cpu_mem_usage=True
                                                          )

processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
# prepare image and text prompt, using the appropriate prompt template
import cv2
im = cv2.imread("images_for_vlm/scene1.png")
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

image = Image.fromarray(im, mode='RGB')
pt = ("You are a robot navigating within an 3-D environment as shown. In the image you see, there are "
                  f"{4} labeled objects. You will be asked to analyze the spatial position of these labeled "
                  f"objects with relation to each other. The red dots on each object are the object's center of mass, "
                  f"which you should use when comparing the position of two objects. From your point of view, "
                  f"answer each question with the"
                  f"descriptors right/left, above/below, in front/behind. If there is no clear spatial difference "
                  f"along a given axis, you can answer 'neutral'")
pt += ("\n1. Where is the led tv in relation to the couch?\n2. Where is the couch in relation to the chandelier?\n3. Where is the chandelier in relation to the kitchen countertop item?")

pt +=  ("\nReason through the task  and describe the 3d layout of the image you see. Output your thinking and at the very end output "
                   "'ANSWER' followed by a json object in the following example format:"
                   "\n{1: ['right', 'above', 'neutral'], 2: ['left', 'neutral', 'in front']}\nThere should be 3 key-value pairs corresponding to the 3 questions")


prompt = f"[INST] <image>\n{pt} [/INST]"
#prompt = f"A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions. USER: <image>\n{pt} ASSISTANT:"

inputs = processor(prompt, image, return_tensors="pt").to(model.device)

print('starting output')
t = time.time()
input_tokens = inputs['input_ids'].shape[1]
print(f'prompt tokens: {input_tokens}')
output = model.generate(**inputs, max_new_tokens=1000)
duration = time.time() - t
print(f'finished output, took {duration} seconds')
output_tokens = output.shape[1]
print(f'output tokens: {output_tokens-input_tokens}')
print(f'efficiency, {(output_tokens-input_tokens)/duration} tokens per second')
print(processor.decode(output[0][input_tokens:], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


starting output
prompt tokens: 274


KeyboardInterrupt: 

In [ ]:
print(prompt)

[INST] <image>
You are a robot navigating within an 3-D environment as shown. In the image you see, there are 4 labeled objects. You will be asked to analyze the spatial position of these labeled objects with relation to each other. The red dots on each object are the object's center of mass, which you should use when comparing the position of two objects. From your point of view, answer each question with thedescriptors right/left, above/below, in front/behind. If there is no clear spatial difference along a given axis, you can answer 'neutral'
1. Where is the led tv in relation to the couch?
2. Where is the couch in relation to the chandelier?
3. Where is the chandelier in relation to the kitchen countertop item?
Reason through the task  and describe the 3d layout of the image you see. Output your thinking and at the very end output 'ANSWER' followed by a json object in the following example format:
{1: ['right', 'above', 'neutral'], 2: ['left', 'neutral', 'in front']}
There should b

In [ ]:

print(type(im))

<class 'numpy.ndarray'>


In [ ]:
im.shape

(3, 1175, 3)

ImportError: cannot import name 'LLaVaAgent' from 'llavaAgent' (/home/dylangoetting/SpatialBenchmark/llavaAgent.py)